# Final Project - Reinforcements Learning 
Hello dear students,<br> this is the template notebook. Please click on the "File" tab and then on "Save a copy into drive".

---
<br>

### Name and ID:
Student 1: Avraham Raviv, 204355390
<br>
Student 2: Yevgeni Berkovitch, 317079234
<br><br>
<img src="https://play-lh.googleusercontent.com/e_oKlKPISbgdzut1H9opevS7-LTB8-8lsmpCdMkhlnqFenZhpjxbLmx7l158-xQQCIY">

### https://github.com/mpSchrader/gym-sokoban

# Installs

In [1]:
%%capture
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install gym
!pip install pygame
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym_sokoban

!imageio_download_bin ffmpeg

# Imports

In [2]:
import random
import time

import numpy as np
import matplotlib.pyplot as plt

import base64
import imageio
from pyvirtualdisplay import Display
from IPython.display import HTML

import gym
from gym import error, spaces, utils
from soko_pap import *

from collections import deque
from queue import PriorityQueue

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten

In [3]:
%matplotlib inline

In [4]:
imageio.plugins.ffmpeg.download()

In [5]:
from gym import logger as gymlogger
gymlogger.set_level(40) # error only

# Display utils
The cell below contains the video display configuration. No need to make changes here.

In [6]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return HTML(tag)

# Utils

In [7]:
def get_distances(room_state):
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):
            if room_state[i][j] == 2:
                target = (i, j)

    distances = np.zeros(shape=room_state.shape)
    visited_cells = set()
    cell_queue = deque()

    visited_cells.add(target)
    cell_queue.appendleft(target)

    while len(cell_queue) != 0:
        cell = cell_queue.pop()
        distance = distances[cell[0]][cell[1]]
        for x,y in ((1,0), (-1,-0), (0,1), (0,-1)):
            next_cell_x, next_cell_y = cell[0]+x, cell[1]+y
            if room_state[next_cell_x][next_cell_y] != 0 and not (next_cell_x, next_cell_y) in visited_cells:
                distances[next_cell_x][next_cell_y] = distance + 1
                visited_cells.add((next_cell_x, next_cell_y))
                cell_queue.appendleft((next_cell_x, next_cell_y))
                
    return distances   

def calc_distances(room_state, distances):
    box = None
    mover = None
    for i in range(room_state.shape[0]):
        for j in range(room_state.shape[1]):            
            if room_state[i][j] == 4:
                box = (i,j)
            
            if room_state[i][j] == 5:
                mover = (i,j)
    
    return mover, box, distances[box[0]][box[1]]   

def box2target_change_reward(room_state, next_room_state, distances):
    if np.array_equal(room_state, next_room_state):
        return -0.5
    
    mover, box, t2b = calc_distances(room_state, distances)
    n_mover, n_box, n_t2b = calc_distances(next_room_state, distances)
    
    change_reward = 0.0
    if n_t2b < t2b:
        change_reward += 1.0
    elif n_t2b > t2b:
        change_reward -= 1.0
        
    m2b = np.sqrt((mover[0]-box[0])**2 + (mover[1]-box[1])**2)
    n_m2b = np.sqrt((n_mover[0]-n_box[0])**2 + (n_mover[1]-n_box[1])**2)
    
    if n_m2b < m2b and m2b >= 2:
        change_reward += 0.25
    elif n_m2b > m2b and n_m2b >= 2:
        change_reward -= 0.25
        
    return change_reward   

# Solution

In [33]:
class SOK_Agent:
    def __init__(self):
        # Construct DQN models
        self.state_size = (5, 5, 1) 
        self.action_size = 8
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.batch_size = 8
        
        # Replay buffers
        self.replay_buffer = deque(maxlen=1500)
        self.prioritized_replay_buffer = deque(maxlen=500)
        self.prioritized_replay_batch = 20        
        
        # Hyperparameters
        self.gamma = 0.9
        self.epsilon = 0.5   
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.99995
        self.replay_rate = 10
        self.update_beta = 0.99

        self.verbosity = 100 
        
        self.action_rotation_map = {
            0: 2,
            1: 3,
            2: 1,
            3: 0,
            4: 6,
            5: 7,
            6: 5,
            7: 4
        }

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=self.state_size))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer="adam") 
        return model

    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append([state, action, reward, next_state, done])    
        
    def copy_to_prioritized_buffer(self, n):
        for i in range(n):
            self.prioritized_replay_buffer.append(self.replay_buffer[-1-i])  

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0]) 

    def replay(self):
        if len(agent.prioritized_replay_buffer) < self.batch_size//2:
            minibatch = random.sample(self.replay_buffer, self.batch_size) 
        else:  
            minibatch = random.sample(self.replay_buffer, self.batch_size//2) 
            minibatch.extend(random.sample(self.prioritized_replay_buffer, self.batch_size//2))
        
        states = np.zeros((self.batch_size*4, self.state_size[0], self.state_size[1]))
        actions = np.zeros(self.batch_size*4, dtype=int)
        rewards = np.zeros(self.batch_size*4)
        next_states = np.zeros((self.batch_size*4, self.state_size[0], self.state_size[1]))
        statuses = np.zeros(self.batch_size*4)
        targets = np.zeros((self.batch_size*4, self.action_size)) 
        
        for i, (state, action, reward, next_state, done) in enumerate(minibatch): 
            for rot in range(4):  
                ind = i*4+rot
                if rot != 0:
                    state = np.rot90(state, axes=(1,2))
                    next_state = np.rot90(next_state, axes=(1,2))
                    action = self.action_rotation_map.get(action)

                states[ind] = state.copy()
                actions[ind] = action
                rewards[ind] = reward
                next_states[ind] = next_state.copy()
                statuses[ind] = 1 if done else 0        

        targets = self.model.predict(states) 
        max_actions = np.argmax(self.model.predict(next_states), axis=1)
        next_rewards = self.target_model.predict(next_states)
        
        ind = 0
        for action, reward, next_reward, max_action, done in zip(actions, rewards, next_rewards, max_actions, statuses):  
            if not done:
                reward += self.gamma * next_reward[max_action]
            targets[ind][action] = reward
            ind += 1
        
        self.model.fit(states, targets, epochs=1, verbose=0)
                
    def update_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon * self.epsilon_decay
        
    def update_target_model(self):
        model_w = self.model.get_weights()
        target_model_w = self.target_model.get_weights()
        updated_target_model_w = []
        for i in range(len(model_w)):
            updated_target_model_w.append(self.update_beta*target_model_w[i] + (1-self.update_beta)*model_w[i])
        self.target_model.set_weights(updated_target_model_w)    
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [34]:
def process_frame(frame):
    f = frame[16:96, 16:96, 0]   
    f = f.reshape(5, 16, 5, 16).max(axis=(1, 3))
    #f = f.flatten()
    f = f / 255
    return np.expand_dims(f, axis=0)

## Training Set

In [40]:
train_episodes = [
    (12, [24, 95]),
    (11, [49, 65]),
    (9,  [2,  86]),
    (8,  [26, 38, 48, 68]),
    (7,  [5,  32, 44, 67]), 
    (6,  [6,  13, 18, 33, 34, 41, 42, 50, 69, 78, 81]),
    (5,  [9,  14, 19, 23, 27, 31, 45, 47, 54, 60, 66, 77, 79, 82, 85]),
    (4,  [28, 59, 62]),
    (3,  [0,  1,  7,  10, 21, 25, 30, 36, 40, 57, 58, 61, 64, 70, 74, 75, 83, 88, 89, 91, 92, 93, 94, 96, 97, 99]), 
    (2,  [8,  11, 12, 35, 43, 46, 71, 72, 73, 80, 84]),
    (1,  [3,  4,  15, 16, 17, 20, 22, 29, 37, 39, 51, 52, 53, 55, 56, 63, 76, 87, 90, 98])    
]

train_episodes = [
    (12, [24, 95]),
    (11, [49, 65]),
    (9,  [2,  86]),
    (8,  [26, 38, 48, 68]),
    (7,  [5,  32, 44, 67]), 
    (6,  [6,  13, 18, 33]),
    (5,  [9,  14, 19, 23, 27]),
    (4,  [28, 59, 62]),
    (3,  [0,  1,  7,  10]), 
    (2,  [8,  11, 12, 35]),
    (1,  [3,  4,  15, 16])    
]

train_list = []
for tep in train_episodes[::-1]:
    train_list.extend(tep[1])

## Training

In [41]:
max_steps = 100

def init_sok(e, tn, continuous_successes):
    test_num = e%tn
    r = train_list[test_num]
    print("Episode %d: Tests %d Successes %d" % (e, tn, continuous_successes))
    random.seed(r)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(max_steps)
    return sok

In [43]:
agent = SOK_Agent()

tn = 1
e = 0

continuous_successes = 0
total_steps = 0
steps_per_episode = []

while tn <= 100:  
    e += 1
    sok = init_sok(e, tn, continuous_successes)
    state = process_frame(sok.get_image('rgb_array'))
    random.seed(e)
    
    room_state = sok.room_state.copy() 
    distances = get_distances(room_state)
    
    for step in range(sok.max_steps):
        total_steps += 1
        current_epsilon = agent.epsilon
        if current_epsilon < 0.1 and step >= 20:
            agent.epsilon = 0.9 
        action = agent.act(state)
        agent.epsilon = current_epsilon
        
        if action < 4:
            next_state, reward, done, _ = sok.step(action+1) 
        else:
            next_state, reward, done, _ = sok.step(action+5)         
        
        next_state = process_frame(next_state)        
        next_room_state = sok.room_state
        
        if not done:
            reward += box2target_change_reward(room_state, next_room_state, distances)
        
        agent.remember(state, action, reward, next_state, done)
        
        state = next_state.copy() 
        room_state = next_room_state.copy()
                
        if e >= 10:            
            if (total_steps+1) % agent.replay_rate == 0:
                agent.replay() 
                agent.update_target_model()
                agent.update_epsilon()
        
        if done:            
            if 3 in sok.room_state:
                continuous_successes += 1
                print("SOLVED! Steps: %d" % (step+1)) 
                agent.copy_to_prioritized_buffer(min(agent.prioritized_replay_batch, step+1))
            else:
                continuous_successes = 0
                
            steps_per_episode.append(step+1)
            #agent.save("saved_models\episode%d.h5" % (e))
            
            break
            
    if continuous_successes >= tn*5:
        tn += 1   
        continuous_successes = 0
        agent.epsilon = max(agent.epsilon, 0.5)

Episode 1: Tests 1 Successes 0
SOLVED! Steps: 1
Episode 2: Tests 1 Successes 1
SOLVED! Steps: 1
Episode 3: Tests 1 Successes 2
SOLVED! Steps: 1
Episode 4: Tests 1 Successes 3
SOLVED! Steps: 1
Episode 5: Tests 1 Successes 4
SOLVED! Steps: 1
Episode 6: Tests 2 Successes 0
SOLVED! Steps: 1
Episode 7: Tests 2 Successes 1
Episode 8: Tests 2 Successes 0
SOLVED! Steps: 1
Episode 9: Tests 2 Successes 1
Episode 10: Tests 2 Successes 0
SOLVED! Steps: 1
Episode 11: Tests 2 Successes 1
Episode 12: Tests 2 Successes 0
SOLVED! Steps: 1
Episode 13: Tests 2 Successes 1
Episode 14: Tests 2 Successes 0
SOLVED! Steps: 1
Episode 15: Tests 2 Successes 1
SOLVED! Steps: 59
Episode 16: Tests 2 Successes 2
SOLVED! Steps: 1
Episode 17: Tests 2 Successes 3
SOLVED! Steps: 1
Episode 18: Tests 2 Successes 4
SOLVED! Steps: 1
Episode 19: Tests 2 Successes 5
SOLVED! Steps: 1
Episode 20: Tests 2 Successes 6
SOLVED! Steps: 1
Episode 21: Tests 2 Successes 7
SOLVED! Steps: 1
Episode 22: Tests 2 Successes 8
SOLVED! Steps: 

SOLVED! Steps: 1
Episode 171: Tests 7 Successes 6
SOLVED! Steps: 1
Episode 172: Tests 7 Successes 7
SOLVED! Steps: 2
Episode 173: Tests 7 Successes 8
SOLVED! Steps: 2
Episode 174: Tests 7 Successes 9
SOLVED! Steps: 2
Episode 175: Tests 7 Successes 10
SOLVED! Steps: 1
Episode 176: Tests 7 Successes 11
SOLVED! Steps: 1
Episode 177: Tests 7 Successes 12
SOLVED! Steps: 1
Episode 178: Tests 7 Successes 13
SOLVED! Steps: 1
Episode 179: Tests 7 Successes 14
SOLVED! Steps: 2
Episode 180: Tests 7 Successes 15
SOLVED! Steps: 2
Episode 181: Tests 7 Successes 16
SOLVED! Steps: 2
Episode 182: Tests 7 Successes 17
SOLVED! Steps: 1
Episode 183: Tests 7 Successes 18
SOLVED! Steps: 1
Episode 184: Tests 7 Successes 19
SOLVED! Steps: 1
Episode 185: Tests 7 Successes 20
SOLVED! Steps: 1
Episode 186: Tests 7 Successes 21
SOLVED! Steps: 2
Episode 187: Tests 7 Successes 22
SOLVED! Steps: 2
Episode 188: Tests 7 Successes 23
SOLVED! Steps: 2
Episode 189: Tests 7 Successes 24
SOLVED! Steps: 1
Episode 190: Tests

SOLVED! Steps: 1
Episode 337: Tests 9 Successes 4
SOLVED! Steps: 2
Episode 338: Tests 9 Successes 5
SOLVED! Steps: 2
Episode 339: Tests 9 Successes 6
SOLVED! Steps: 2
Episode 340: Tests 9 Successes 7
Episode 341: Tests 9 Successes 0
Episode 342: Tests 9 Successes 0
SOLVED! Steps: 1
Episode 343: Tests 9 Successes 1
SOLVED! Steps: 1
Episode 344: Tests 9 Successes 2
SOLVED! Steps: 1
Episode 345: Tests 9 Successes 3
SOLVED! Steps: 1
Episode 346: Tests 9 Successes 4
SOLVED! Steps: 2
Episode 347: Tests 9 Successes 5
SOLVED! Steps: 2
Episode 348: Tests 9 Successes 6
SOLVED! Steps: 2
Episode 349: Tests 9 Successes 7
SOLVED! Steps: 13
Episode 350: Tests 9 Successes 8
Episode 351: Tests 9 Successes 0
SOLVED! Steps: 1
Episode 352: Tests 9 Successes 1
SOLVED! Steps: 1
Episode 353: Tests 9 Successes 2
SOLVED! Steps: 1
Episode 354: Tests 9 Successes 3
SOLVED! Steps: 1
Episode 355: Tests 9 Successes 4
SOLVED! Steps: 2
Episode 356: Tests 9 Successes 5
SOLVED! Steps: 2
Episode 357: Tests 9 Successes 6


SOLVED! Steps: 1
Episode 508: Tests 9 Successes 4
SOLVED! Steps: 2
Episode 509: Tests 9 Successes 5
SOLVED! Steps: 2
Episode 510: Tests 9 Successes 6
SOLVED! Steps: 2
Episode 511: Tests 9 Successes 7
SOLVED! Steps: 2
Episode 512: Tests 9 Successes 8
Episode 513: Tests 9 Successes 0
SOLVED! Steps: 1
Episode 514: Tests 9 Successes 1
SOLVED! Steps: 1
Episode 515: Tests 9 Successes 2
SOLVED! Steps: 1
Episode 516: Tests 9 Successes 3
SOLVED! Steps: 1
Episode 517: Tests 9 Successes 4
SOLVED! Steps: 2
Episode 518: Tests 9 Successes 5
SOLVED! Steps: 2
Episode 519: Tests 9 Successes 6
SOLVED! Steps: 2
Episode 520: Tests 9 Successes 7
SOLVED! Steps: 2
Episode 521: Tests 9 Successes 8
Episode 522: Tests 9 Successes 0
SOLVED! Steps: 1
Episode 523: Tests 9 Successes 1
SOLVED! Steps: 1
Episode 524: Tests 9 Successes 2
SOLVED! Steps: 1
Episode 525: Tests 9 Successes 3
SOLVED! Steps: 1
Episode 526: Tests 9 Successes 4
SOLVED! Steps: 2
Episode 527: Tests 9 Successes 5
SOLVED! Steps: 2
Episode 528: Test

SOLVED! Steps: 1
Episode 679: Tests 9 Successes 4
SOLVED! Steps: 2
Episode 680: Tests 9 Successes 5
SOLVED! Steps: 2
Episode 681: Tests 9 Successes 6
SOLVED! Steps: 2
Episode 682: Tests 9 Successes 7
SOLVED! Steps: 2
Episode 683: Tests 9 Successes 8
Episode 684: Tests 9 Successes 0
SOLVED! Steps: 1
Episode 685: Tests 9 Successes 1
SOLVED! Steps: 1
Episode 686: Tests 9 Successes 2
SOLVED! Steps: 1
Episode 687: Tests 9 Successes 3
SOLVED! Steps: 1
Episode 688: Tests 9 Successes 4
SOLVED! Steps: 2
Episode 689: Tests 9 Successes 5
SOLVED! Steps: 2
Episode 690: Tests 9 Successes 6
SOLVED! Steps: 2
Episode 691: Tests 9 Successes 7
SOLVED! Steps: 2
Episode 692: Tests 9 Successes 8
Episode 693: Tests 9 Successes 0
SOLVED! Steps: 1
Episode 694: Tests 9 Successes 1
SOLVED! Steps: 1
Episode 695: Tests 9 Successes 2
SOLVED! Steps: 1
Episode 696: Tests 9 Successes 3
SOLVED! Steps: 1
Episode 697: Tests 9 Successes 4
SOLVED! Steps: 2
Episode 698: Tests 9 Successes 5
SOLVED! Steps: 2
Episode 699: Test

...

SOLVED! Steps: 3
Episode 20846: Tests 15 Successes 11
SOLVED! Steps: 3
Episode 20847: Tests 15 Successes 12
SOLVED! Steps: 4
Episode 20848: Tests 15 Successes 13
Episode 20849: Tests 15 Successes 0
Episode 20850: Tests 15 Successes 0
SOLVED! Steps: 1
Episode 20851: Tests 15 Successes 1
SOLVED! Steps: 1
Episode 20852: Tests 15 Successes 2
SOLVED! Steps: 1
Episode 20853: Tests 15 Successes 3
SOLVED! Steps: 1
Episode 20854: Tests 15 Successes 4
SOLVED! Steps: 2
Episode 20855: Tests 15 Successes 5
SOLVED! Steps: 2
Episode 20856: Tests 15 Successes 6
SOLVED! Steps: 2
Episode 20857: Tests 15 Successes 7
SOLVED! Steps: 2
Episode 20858: Tests 15 Successes 8
SOLVED! Steps: 3
Episode 20859: Tests 15 Successes 9
SOLVED! Steps: 3
Episode 20860: Tests 15 Successes 10
SOLVED! Steps: 3
Episode 20861: Tests 15 Successes 11
SOLVED! Steps: 3
Episode 20862: Tests 15 Successes 12
SOLVED! Steps: 4
Episode 20863: Tests 15 Successes 13
Episode 20864: Tests 15 Successes 0
Episode 20865: Tests 15 Successes 0
S

SOLVED! Steps: 2
Episode 21007: Tests 15 Successes 7
SOLVED! Steps: 2
Episode 21008: Tests 15 Successes 8
SOLVED! Steps: 3
Episode 21009: Tests 15 Successes 9
SOLVED! Steps: 3
Episode 21010: Tests 15 Successes 10
SOLVED! Steps: 3
Episode 21011: Tests 15 Successes 11
SOLVED! Steps: 3
Episode 21012: Tests 15 Successes 12
SOLVED! Steps: 4
Episode 21013: Tests 15 Successes 13
Episode 21014: Tests 15 Successes 0
Episode 21015: Tests 15 Successes 0
SOLVED! Steps: 1
Episode 21016: Tests 15 Successes 1
SOLVED! Steps: 1
Episode 21017: Tests 15 Successes 2
SOLVED! Steps: 1
Episode 21018: Tests 15 Successes 3
SOLVED! Steps: 1
Episode 21019: Tests 15 Successes 4
SOLVED! Steps: 2
Episode 21020: Tests 15 Successes 5
SOLVED! Steps: 2
Episode 21021: Tests 15 Successes 6
SOLVED! Steps: 2
Episode 21022: Tests 15 Successes 7
SOLVED! Steps: 2
Episode 21023: Tests 15 Successes 8
SOLVED! Steps: 3
Episode 21024: Tests 15 Successes 9
SOLVED! Steps: 3
Episode 21025: Tests 15 Successes 10
SOLVED! Steps: 3
Episo

SOLVED! Steps: 1
Episode 21168: Tests 15 Successes 3
SOLVED! Steps: 1
Episode 21169: Tests 15 Successes 4
SOLVED! Steps: 2
Episode 21170: Tests 15 Successes 5
SOLVED! Steps: 2
Episode 21171: Tests 15 Successes 6
SOLVED! Steps: 2
Episode 21172: Tests 15 Successes 7
SOLVED! Steps: 2
Episode 21173: Tests 15 Successes 8
SOLVED! Steps: 3
Episode 21174: Tests 15 Successes 9
SOLVED! Steps: 3
Episode 21175: Tests 15 Successes 10
SOLVED! Steps: 3
Episode 21176: Tests 15 Successes 11
SOLVED! Steps: 3
Episode 21177: Tests 15 Successes 12
SOLVED! Steps: 4
Episode 21178: Tests 15 Successes 13
Episode 21179: Tests 15 Successes 0
Episode 21180: Tests 15 Successes 0
SOLVED! Steps: 1
Episode 21181: Tests 15 Successes 1
SOLVED! Steps: 1
Episode 21182: Tests 15 Successes 2
SOLVED! Steps: 1
Episode 21183: Tests 15 Successes 3
SOLVED! Steps: 1
Episode 21184: Tests 15 Successes 4
SOLVED! Steps: 2
Episode 21185: Tests 15 Successes 5
SOLVED! Steps: 2
Episode 21186: Tests 15 Successes 6
SOLVED! Steps: 2
Episod

Episode 21329: Tests 15 Successes 0
Episode 21330: Tests 15 Successes 0
SOLVED! Steps: 1
Episode 21331: Tests 15 Successes 1
SOLVED! Steps: 1
Episode 21332: Tests 15 Successes 2
SOLVED! Steps: 1
Episode 21333: Tests 15 Successes 3
SOLVED! Steps: 1
Episode 21334: Tests 15 Successes 4
SOLVED! Steps: 2
Episode 21335: Tests 15 Successes 5
SOLVED! Steps: 2
Episode 21336: Tests 15 Successes 6
SOLVED! Steps: 2
Episode 21337: Tests 15 Successes 7
SOLVED! Steps: 2
Episode 21338: Tests 15 Successes 8
SOLVED! Steps: 3
Episode 21339: Tests 15 Successes 9
SOLVED! Steps: 3
Episode 21340: Tests 15 Successes 10
SOLVED! Steps: 3
Episode 21341: Tests 15 Successes 11
SOLVED! Steps: 3
Episode 21342: Tests 15 Successes 12
SOLVED! Steps: 4
Episode 21343: Tests 15 Successes 13
Episode 21344: Tests 15 Successes 0
Episode 21345: Tests 15 Successes 0
SOLVED! Steps: 1
Episode 21346: Tests 15 Successes 1
SOLVED! Steps: 1
Episode 21347: Tests 15 Successes 2
SOLVED! Steps: 1
Episode 21348: Tests 15 Successes 3
SOLV

KeyboardInterrupt: 

## Test Generalization

#### Learned Policy

In [44]:
agent.epsilon = 0.0
num_solved = 0

for t in range(100):    
    random.seed(t+100)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(100)
    
    steps = 0
        
    state = sok.get_image('rgb_array')
    done = False
    while not done:
        steps += 1
        action = agent.act(process_frame(state))

        if action < 4:
            action += 1
        else:
            action += 5

        state, reward, done, info = sok.step(action)
        
    solved = False
    if 3 in sok.room_state:
        solved = True
        num_solved += 1
    
    print("Puzzle # %d: %d (%s) [%d/%d]" % ((t+1), steps, "YES" if solved else "NOT", num_solved, (t+1)))

Puzzle # 1: 100 (NOT) [0/1]
Puzzle # 2: 100 (NOT) [0/2]
Puzzle # 3: 100 (NOT) [0/3]
Puzzle # 4: 100 (NOT) [0/4]
Puzzle # 5: 100 (NOT) [0/5]
Puzzle # 6: 100 (NOT) [0/6]
Puzzle # 7: 100 (NOT) [0/7]
Puzzle # 8: 2 (YES) [1/8]
Puzzle # 9: 100 (NOT) [1/9]
Puzzle # 10: 100 (NOT) [1/10]
Puzzle # 11: 100 (NOT) [1/11]
Puzzle # 12: 100 (NOT) [1/12]
Puzzle # 13: 100 (NOT) [1/13]
Puzzle # 14: 100 (NOT) [1/14]
Puzzle # 15: 100 (NOT) [1/15]
Puzzle # 16: 100 (NOT) [1/16]
Puzzle # 17: 100 (NOT) [1/17]
Puzzle # 18: 100 (NOT) [1/18]
Puzzle # 19: 100 (NOT) [1/19]
Puzzle # 20: 3 (YES) [2/20]
Puzzle # 21: 3 (YES) [3/21]
Puzzle # 22: 100 (NOT) [3/22]
Puzzle # 23: 100 (NOT) [3/23]
Puzzle # 24: 100 (NOT) [3/24]
Puzzle # 25: 100 (NOT) [3/25]
Puzzle # 26: 100 (NOT) [3/26]
Puzzle # 27: 100 (NOT) [3/27]
Puzzle # 28: 100 (NOT) [3/28]
Puzzle # 29: 100 (NOT) [3/29]
Puzzle # 30: 1 (YES) [4/30]
Puzzle # 31: 100 (NOT) [4/31]
Puzzle # 32: 3 (YES) [5/32]
Puzzle # 33: 3 (YES) [6/33]
Puzzle # 34: 100 (NOT) [6/34]
Puzzle # 3

#### Random Policy

In [ ]:
agent.epsilon = 1.0
num_solved = 0

for t in range(100):    
    random.seed(t+100)
    sok = PushAndPullSokobanEnv(dim_room=(7, 7), num_boxes=1)
    sok.set_maxsteps(100)
    
    steps = 0
        
    state = sok.get_image('rgb_array')
    done = False
    while not done:
        steps += 1
        action = agent.act(process_frame(state))

        if action < 4:
            action += 1
        else:
            action += 5

        state, reward, done, info = sok.step(action)
        
    solved = False
    if 3 in sok.room_state:
        solved = True
        num_solved += 1
    
    print("Puzzle # %d: %d (%s) [%d/%d]" % ((t+1), steps, "YES" if solved else "NOT", num_solved, (t+1)))

Puzzle # 1: 100 (NOT) [0/1]
Puzzle # 2: 100 (NOT) [0/2]
Puzzle # 3: 100 (NOT) [0/3]
Puzzle # 4: 100 (NOT) [0/4]
Puzzle # 5: 100 (NOT) [0/5]
Puzzle # 6: 100 (NOT) [0/6]
Puzzle # 7: 100 (NOT) [0/7]
Puzzle # 8: 46 (YES) [1/8]
Puzzle # 9: 26 (YES) [2/9]
Puzzle # 10: 100 (NOT) [2/10]
Puzzle # 11: 9 (YES) [3/11]
Puzzle # 12: 100 (NOT) [3/12]
Puzzle # 13: 100 (NOT) [3/13]
Puzzle # 14: 100 (NOT) [3/14]
Puzzle # 15: 1 (YES) [4/15]
Puzzle # 16: 100 (NOT) [4/16]
Puzzle # 17: 100 (NOT) [4/17]
Puzzle # 18: 100 (NOT) [4/18]
Puzzle # 19: 89 (YES) [5/19]
Puzzle # 20: 10 (YES) [6/20]
Puzzle # 21: 100 (NOT) [6/21]
Puzzle # 22: 4 (YES) [7/22]
Puzzle # 23: 100 (NOT) [7/23]
Puzzle # 24: 100 (NOT) [7/24]
Puzzle # 25: 100 (NOT) [7/25]
Puzzle # 26: 100 (NOT) [7/26]
Puzzle # 27: 100 (NOT) [7/27]
Puzzle # 28: 100 (NOT) [7/28]
Puzzle # 29: 100 (NOT) [7/29]
Puzzle # 30: 11 (YES) [8/30]
Puzzle # 31: 100 (NOT) [8/31]
Puzzle # 32: 100 (NOT) [8/32]
Puzzle # 33: 100 (NOT) [8/33]
Puzzle # 34: 100 (NOT) [8/34]
Puzzle # 